In [1]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaForCausalLM, LlamaTokenizer
from huggingface_hub import login
import os
import torch
import transformers
import random

[2023-08-16 20:02:55,544] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
# global configs
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
model_name_or_path = "/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9" # "meta-llama/Llama-2-7b-hf"
tokenizer_name_or_path = "/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289bee767f0771166b5917e70e470"
gpu_id = 3

In [3]:
device = f"cuda:{gpu_id}"

In [4]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
tokenizer = LlamaTokenizer.from_pretrained(tokenizer_name_or_path)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [5]:
print(tokenizer)
print(tokenizer.special_tokens_map)

LlamaTokenizer(name_or_path='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289bee767f0771166b5917e70e470', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=False)
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}


In [6]:
# no default pad token for llama!
# here we add all special tokens again, because the default ones are not in the special_tokens_map

num_added_tokens = tokenizer.add_special_tokens(
    {
        "bos_token": "<s>",
        "eos_token": "</s>",
        "unk_token": "<unk>",
        "pad_token": "<pad>",
    }
)
assert num_added_tokens in [
    0,
    1,
], "LlamaTokenizer should only add one special token - the pad_token, or no tokens if pad token present."


In [7]:
print(tokenizer)
print(tokenizer.special_tokens_map)

LlamaTokenizer(name_or_path='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289bee767f0771166b5917e70e470', vocab_size=32000, model_max_length=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}


In [8]:
model = LlamaForCausalLM.from_pretrained(model_name_or_path, low_cpu_mem_usage=True)
model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [9]:
print(model)
print(model.config)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [10]:
embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))

In [11]:
print(model.generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}



In [30]:
def sparse_print(*arg, **kwargs):
    print(*arg, **kwargs, sep="\n", end="\n\n")
    
def generate(prompt_or_list, generation_config):
    if not isinstance(generation_config, transformers.GenerationConfig):
        if isinstance(generation_config, dict):
            generation_config = transformers.GenerationConfig.from_dict(generation_config)
    
    inputs = tokenizer(prompt_or_list, 
                       padding=True,
                       return_tensors="pt"
                       ).to(device)
    sparse_print("inputs:")
    sparse_print(inputs)

    outputs = model.generate(**inputs, generation_config=generation_config)
    sparse_print(outputs)
    
    if not generation_config.return_dict_in_generate:
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        sparse_print(decoded_outputs)
        
        return outputs
    else: # generation_config.return_dict_in_generate == True
        outputs_len = outputs["sequences"].shape[0]

            
        example_idx = random.randint(0, outputs_len-1)
        sparse_print(f"example_idx = {example_idx}")

        if outputs_len > 5:
            example_decoded_output = tokenizer.decode(outputs["sequences"][example_idx], skip_special_tokens=True)
            sparse_print(example_decoded_output)
        else: # outputs_len <= 5
            decoded_outputs = tokenizer.batch_decode(outputs["sequences"], skip_special_tokens=True)
            sparse_print(decoded_outputs)
        
        if generation_config.output_scores:
            example_scores = outputs["scores"][example_idx]
            sparse_print(example_scores.shape)
            sparse_print(example_scores)
        
            example_probs = torch.nn.functional.softmax(example_scores, dim=-1)
            sparse_print(example_probs[example_idx])
            sparse_print(sum(example_probs[example_idx]))

            # slow cpu
            # for idx, prob in enumerate(first_probs[0]):
            #     if prob > 1e-5:
            #         print(f"token_id: {idx}, token: {tokenizer.decode([idx])}, prob: {prob}")

            # fast gpu
            # non_zero_idxs = torch.gt(first_probs.view(-1),1e-1)
            threshold = 0.1
            mask = example_probs > threshold
            non_zero_idxs = torch.nonzero(mask)
            sparse_print(non_zero_idxs)
            non_zero_vals = example_probs.view(-1)[non_zero_idxs]
            sparse_print(non_zero_vals)

        return outputs

In [41]:
# test
generation_config = {
    "do_sample": True,
    # "temperature": 0.8,
    # "top_p": 0.95,
    "temperature": 0.7,
    "top_p": 0.9,
    "max_new_tokens": 10,
    "return_dict_in_generate": True,
}

generation_config = transformers.GenerationConfig.from_dict(generation_config)

prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
 
outputs = generate(prompt_or_list=prompts, generation_config=generation_config)

decoded_outputs = tokenizer.batch_decode(**outputs, skip_special_tokens=True)

for prompt, decoded_output in zip(prompts, decoded_outputs):
    print(f"Prompt    : {prompt}")
    print(f"Generation: {decoded_output[len(prompt):]}")
    print()

inputs:

{'input_ids': tensor([[    1, 15043, 29892,   590,  1024,   338, 32000, 32000],
        [    1,   450,  6673,   310,   278,  3303,  3900,   338],
        [    1,   450,  7483,   310,  3444,   338, 32000, 32000],
        [    1,   450,  5434,   310,   319, 29902,   338, 32000]],
       device='cuda:3'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0]], device='cuda:3')}

SampleDecoderOnlyOutput(sequences=tensor([[    1, 15043, 29892,   590,  1024,   338, 32000, 32000, 30488, 30488,
         30488, 30488, 30488, 30488, 30488, 30488, 30488, 30488],
        [    1,   450,  6673,   310,   278,  3303,  3900,   338,   278,  2343,
           310,  2106,   322,  2343,   310,  5874,   310,   278],
        [    1,   450,  7483,   310,  3444,   338, 32000, 32000, 30488,  3681,
         29889,   450,  4272,   338,  5982,   297,   278,  6641],
        [    1,   450,  5434,   310,   319, 

In [21]:
# debug
print(generation_config.return_dict_in_generate)

False


In [ ]:
problem = "A $90^\\circ$ rotation around the origin in the counter-clockwise direction is applied to $7 + 2i.$  What is the resulting complex number?"
steps = [
    "I know that a $90^\\circ$ rotation around the origin in the complex plane can be achieved by multiplying the complex number by $i$, since $i$ has a magnitude of 1 and an argument of $90^\\circ$.",
    "So, I can write the rotation as $(7 + 2i)i = 7i + 2i^2.$",
    "To simplify this expression, I recall that $i^2 = -1$, so I can substitute that and get $7i - 2.$",
    "This is the resulting complex number after the rotation.\n\n# Answer\n\n7i - 2",
]

In [ ]:
rating2word = {1: "positive", -1: "negative", 0: "neutral"}
rating_words = list(rating2word.values())
rating_token_ids = tokenizer(rating_words, add_special_tokens=False).input_ids
print(rating_token_ids)

# constraint = transformers.Constraint(type="must_include", token="apple")

In [ ]:
# config
generation_config = {
    "do_sample": False,
    "max_new_tokens": 1, 
    "force_words_ids": rating_token_ids,
    "num_beams": 2,
    "remove_invalid_values": True,
    "return_dict_in_generate": True,
    "output_scores": True
}

generation_config = transformers.GenerationConfig.from_dict(generation_config)

{'input_ids': tensor([[    1,   319,   395, 29929, 29900,  3823,  6034, 29938, 13733,  2820,
           278,  3978,   297,   278,  6795, 29899, 13058,  3538,  5305,   338,
          7436,   304,   395, 29955,   718, 29871, 29906, 29875,  7449, 29871,
          1724,   338,   278,  9819,  4280,  1353, 29973]], device='cuda:5'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:5')}
BeamSearchDecoderOnlyOutput(sequences=tensor([[    1,   319,   395, 29929, 29900,  3823,  6034, 29938, 13733,  2820,
           278,  3978,   297,   278,  6795, 29899, 13058,  3538,  5305,   338,
          7436,   304,   395, 29955,   718, 29871, 29906, 29875,  7449, 29871,
          1724,   338,   278,  9819,  4280,  1353, 29973,    13]],
       device='cuda:5'), sequences_scores=tensor([-0.0062], device='cuda:5'), scores=(tensor([[-2.0333e+01, -2.0216e+01, -2.5555e+00,  ..., -2.0804e+01,
         

In [ ]:
generate(problem + steps[0], generation_config=generation_config)